In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
!pip install transformers sentence-transformers faiss-cpu patool underthesea langchain langchain-community langchain-openai

INFO: pip is looking at multiple versions of langchain-community to determine which version is compatible with other requirements. This could take a while.
INFO: pip is looking at multiple versions of langchain-openai to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.4/31.4 MB 77.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 87.9/87.9 kB 7.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.3/8.3 MB 95.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 978.4/978.4 kB 71.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 98.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.0/76.0 kB 7.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 80.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.7/64.7 kB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 5.0 MB/s 

In [4]:
import os
import patoolib
import json
import pandas as pd
import numpy as np
from typing import List, Dict, Tuple
import torch
from transformers import AutoTokenizer, AutoModel
from sentence_transformers import SentenceTransformer
import faiss
from underthesea import word_tokenize
import re
from langchain_openai import OpenAIEmbeddings, ChatOpenAI
from langchain_community.vectorstores import FAISS
from langchain.chains import RetrievalQA
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.docstore.document import Document
from langchain.prompts import PromptTemplate

In [ ]:
base_dir = "/content/drive/MyDrive/VSL"
os.makedirs(f"{base_dir}/data/raw", exist_ok=True)
os.makedirs(f"{base_dir}/data/videos", exist_ok=True)
os.makedirs(f"{base_dir}/data/embeddings", exist_ok=True)

In [ ]:
#extract videos from rar file
rar_path = f"{base_dir}/data/raw/VIDEO.rar"
output_dir = f"{base_dir}/data/videos"
patoolib.extract_archive(rar_path, outdir=output_dir)

INFO patool: Extracting /content/drive/MyDrive/VSL/data/raw/VIDEO.rar ...
INFO:patool:Extracting /content/drive/MyDrive/VSL/data/raw/VIDEO.rar ...
INFO patool: running /usr/bin/unrar x -kb -or -- /content/drive/MyDrive/VSL/data/raw/VIDEO.rar
INFO:patool:running /usr/bin/unrar x -kb -or -- /content/drive/MyDrive/VSL/data/raw/VIDEO.rar
INFO patool: ... /content/drive/MyDrive/VSL/data/raw/VIDEO.rar extracted to `/content/drive/MyDrive/VSL/data/videos'.
INFO:patool:... /content/drive/MyDrive/VSL/data/raw/VIDEO.rar extracted to `/content/drive/MyDrive/VSL/data/videos'.


'/content/drive/MyDrive/VSL/data/videos'

In [ ]:
#prepare data
class VietnameseSignLanguageData:
    def __init__(self, json_path: str):
        self.json_path = json_path
        self.data = None
        self.df = None

    def load_data(self):
        with open(self.json_path, 'r', encoding='utf-8') as f:
            self.data = json.load(f)

        self.df = pd.DataFrame(self.data['data'])
        print(f"Loaded {len(self.df)} sign language entries")
        return self.df

    def preprocess_text(self, text: str) -> str:
        if not text or pd.isna(text):
          return ""
        text = text.lower()
        text = re.sub(r'[^\w\s]', '', text)
        text = re.sub(r'\s+', ' ', text).strip()
        return text

    def create_searchable_text(self, row: pd.Series) -> str:
        parts = [
            row['word'],
            row.get('_word', ''),
            row.get('description', ''),
            row.get('tl', '')
        ]
        return " ".join([str(p) for p in parts if p and not pd.isna(p)])

    def prepare_data(self):
        self.df['word_normalized'] = self.df['word'].apply(self.preprocess_text)
        self.df['description_normalized'] = self.df['description'].apply(self.preprocess_text)
        self.df['searchable_text'] = self.df.apply(self.create_searchable_text, axis=1)
        self.df['searchable_text_normalized'] = self.df['searchable_text'].apply(self.preprocess_text)
        print("Data preprocessing complete")
        return self.df

In [ ]:
#faiss and embedding
class VSLKnowledgeBase:
    def __init__(self, df: pd.DataFrame):
        self.df = df
        self.vectorstore = None
        self.retriever = None

    def build_vectorstore(self):
        embeddings = OpenAIEmbeddings(model="") #text-embedding-3-small

        documents = [
            Document(
                page_content=row["searchable_text_normalized"],
                metadata={
                    "word": row.get("word", ""),
                    "description": row.get("description", ""),
                    "video_id": row.get("_id", ""),
                    "part_of_speech": row.get("tl", ""),
                    "type": row.get("type", "")
                }
            )
            for _, row in self.df.iterrows()
        ]
        self.vectorstore = FAISS.from_documents(documents, embeddings)
        self.retriever = self.vectorstore.as_retriever(search_kwargs={"k": 3})
        print("FAISS index built successfully!")

    def save(self, path: str):
        self.vectorstore.save_local(path)
        print(f"Vectorstore saved at {path}")

    def load(self, path: str):
        embeddings = OpenAIEmbeddings(model="text-embedding-3-large")
        self.vectorstore = FAISS.load_local(path, embeddings, allow_dangerous_deserialization=True)
        self.retriever = self.vectorstore.as_retriever(search_kwargs={"k": 3})
        print("Vectorstore loaded successfully!")


In [ ]:
class VietnameseSignLanguageChatbot:
    def __init__(self, retriever):
        self.llm = ChatOpenAI(model="", temperature=) #gpt-4o-mini
        self.retriever = retriever

        template = """
        Bạn là trợ lý thông thạo Ngôn ngữ Ký hiệu Việt Nam (VSL).
        Dưới đây là thông tin về các ký hiệu được truy xuất từ cơ sở dữ liệu.

        Ngữ cảnh:
        {context}

        Câu hỏi của người dùng:
        {question}

        Dựa vào ngữ cảnh, mô tả lại cách biểu diễn ký hiệu, nghĩa của từ, và nếu có video thì trả về đường dẫn.
        Trả lời bằng tiếng Việt tự nhiên, dễ hiểu, có thể kèm theo mô tả chi tiết tay, hướng di chuyển, hoặc cảm xúc.
        """

        self.prompt = PromptTemplate(
            input_variables=["context", "question"],
            template=template
        )

        self.chain = RetrievalQA.from_chain_type(
            llm=self.llm,
            chain_type="stuff",
            retriever=self.retriever,
            chain_type_kwargs={"prompt": self.prompt}
        )

    def ask(self, query: str):
        result = self.chain.invoke({"query": query})
        return result["result"]


In [ ]:
#Initialize chatbot
#Prepare data
data_handler = VietnameseSignLanguageData("/content/drive/MyDrive/VSL/data/VSL_DATA.json")
df = data_handler.load_data()
df = data_handler.prepare_data()

#Build or load FAISS index
knowledge_base = VSLKnowledgeBase(df)
knowledge_base.build_vectorstore()
# knowledge_base.save("vsl_faiss_index")

#Initialize chatbot
chatbot = VietnameseSignLanguageChatbot(knowledge_base.retriever)


Loading data...
Loaded 4362 sign language entries
Data preprocessing complete
Initializing embedding model...
Loading SentenceTransformer model: dangvantuan/vietnamese-embedding
Model loaded successfully.
Building knowledge base...


Batches:   0%|          | 0/137 [00:00<?, ?it/s]

Built FAISS index with 4362 vectors
Knowledge base built successfully!


In [ ]:
def interactive_chat(chatbot):
    print("Nhập câu hỏi của bạn (hoặc 'quit' để thoát)")

    while True:
        user_input = input("Bạn: ").strip()

        if user_input.lower() in ['quit', 'exit', 'thoát']:
            print("Cảm ơn bạn đã sử dụng chatbot! Tạm biệt!")
            break

        if not user_input:
            continue

        response = chatbot.chat(user_input, top_k=1)
        print(f"\nChatbot:\n{response}")

interactive_chat(chatbot)

Nhập câu hỏi của bạn (hoặc 'quit' để thoát)
Bạn: từ địa chỉ được diễn tả bằng ngôn ngữ kí hiệu như thế nào 

Chatbot:
Tìm thấy 1 kết quả:

--- Kết quả 1 ---
Từ: nghĩa trang
ID Video: W02371
Mô tả: Nghĩa địa.
Loại từ: Danh từ
Độ tương đồng: 75.30%


Bạn: từ địa chỉ được diễn tả như thế nào

Chatbot:
Tìm thấy 1 kết quả:

--- Kết quả 1 ---
Từ: nghĩa trang
ID Video: W02371
Mô tả: Nghĩa địa.
Loại từ: Danh từ
Độ tương đồng: 65.55%


Bạn: cách diễn tả từ địa chỉ

Chatbot:
Tìm thấy 1 kết quả:

--- Kết quả 1 ---
Từ: nghĩa trang
ID Video: W02371
Mô tả: Nghĩa địa.
Loại từ: Danh từ
Độ tương đồng: 63.80%


Bạn: quit
Cảm ơn bạn đã sử dụng chatbot! Tạm biệt!
